In [17]:
import pandas as pd
import requests
import json
import time
import os


In [17]:
root_path = '../Dataset'

raw_path = os.path.join(root_path, 'Raw')

if not os.path.exists(raw_path):
    os.makedirs(raw_path)

In [6]:
df = pd.read_csv(os.path.join(root_path, 'SO.csv'), encoding='ISO-8859-1')
print(df.dtypes)
print(df.shape)
df.head()

Question_title              object
Question_body               object
Question_answer_count        int64
Question_comment_count       int64
Question_creation_date      object
Question_favorite_count    float64
Question_last_edit_date     object
Question_score               int64
Question_tags               object
Question_view_count          int64
Owner_creation_date         object
Owner_last_access_date      object
Owner_location              object
Owner_reputation             int64
Owner_up_votes               int64
Owner_down_votes             int64
Owner_views                  int64
Answer_body                 object
Answer_comment_count       float64
Answer_creation_date        object
Answer_last_edit_date       object
Answer_score               float64
dtype: object
(11641, 22)


,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_date,Question_favorite_count,Question_last_edit_date,Question_score,Question_tags,Question_view_count,...,Owner_location,Owner_reputation,Owner_up_votes,Owner_down_votes,Owner_views,Answer_body,Answer_comment_count,Answer_creation_date,Answer_last_edit_date,Answer_score
0,How to specify cluster init script for spark Job,<p>My job needs some init scripts to be execut...,1,0,2019-09-17 21:12:25.47 UTC,1.0,NaN,0,spark-structured-streaming|azure-databricks,1188,...,NaN,195,0,0,42,"<p>To fine tune Spark jobs, you can provide cu...",1.0,2019-09-18 10:02:15.633 UTC,NaN,-1.0
1,Moving from Visual Sourcesafe to Mercurial,<p>What's the best way to move a Visual Source...,6,0,2009-06-07 13:22:01.85 UTC,11.0,2012-03-30 12:24:27.683 UTC,19,mercurial|migration|dvcs|visual-sourcesafe,5119,...,"Denver, CO",5591,341,2,421,<p>While I haven't made that particular conver...,0.0,2009-06-07 13:53:26.763 UTC,NaN,14.0
2,Git Submodule to a subfolder,<p>In svn you can link a repository to any fol...,3,0,2009-07-13 18:28:34.28 UTC,5.0,NaN,32,git|dvcs|git-submodules,11984,...,"Cape Town, Western Cape, South Africa",21264,768,8,1337,"<p>Git does not support partial checkouts, so ...",2.0,2009-07-13 19:00:25.293 UTC,2019-12-31 21:22:10.28 UTC,26.0
3,How to pass a request to sagemaker using postman,<p>I've trained a model on sagemaker and have ...,2,0,2018-04-05 15:03:12.383 UTC,2.0,NaN,8,web-services|amazon-web-services|postman|amazo...,6746,...,NaN,1124,156,1,153,"<p>Once the endpoint is created, you can invok...",0.0,2018-04-05 16:37:35.433 UTC,NaN,11.0
4,Verify if a tag was done in the master branch,"<p>In this project I am working, we do deploym...",4,3,2018-04-05 05:56:25.437 UTC,1.0,NaN,8,git|version-control|repository|dvcs|git-tag,6638,...,NaN,1981,1,0,68,"<p>As several people noted, your question cann...",0.0,2018-04-05 15:04:25.657 UTC,NaN,11.0


In [7]:
tags = {'databricks', 'databricks-unity-catalog', 'databricks-ml', 'amazon-sagemaker', 'amazon-sagemaker-experiments', 'mlflow', 'dvc', 'wandb', 'clearml', 'comet-ml', 'neptune', 'pachyderm', 'lakefs'}

In [8]:
df['Question_valid_tags'] = [[] for _ in range(len(df))]
for index, row in df.iterrows():
    row_tags = set(str(row['Question_tags']).strip().split('|'))
    df.at[index, 'Question_valid_tags'] = list(row_tags.intersection(tags))

In [9]:
df0 = df[df['Question_valid_tags'].map(len) == 0]
df0.shape

(3397, 23)

In [10]:
df = df[df['Question_valid_tags'].map(len) > 0]
df.shape

(8244, 23)

In [11]:
df2 = df[df['Question_valid_tags'].map(len) > 1]
df2.shape

(101, 23)

In [12]:
df3 = df[df['Question_valid_tags'].map(len) > 2]
df3.shape

(0, 23)

In [15]:
filter_path = os.path.join(root_path, 'Filter')

if not os.path.isdir(filter_path):
    os.mkdir(filter_path)

df.to_csv(os.path.join(filter_path, 'All_valid_posts.csv'), encoding='ISO-8859-1', index=False)
df0.to_csv(os.path.join(filter_path, 'All_invalid_posts.csv'), encoding='ISO-8859-1', index=False)
df2.to_csv(os.path.join(filter_path, 'All_double_tag_posts.csv'), encoding='ISO-8859-1', index=False)

In [18]:
df0 = df[df['Question_score'] > -1]
df1 = df0[df0['Question_score'] > 0]
df0.to_csv(os.path.join(filter_path, 'All_qualified_posts.csv'), encoding='ISO-8859-1', index=False)
df1.to_csv(os.path.join(filter_path, 'All_great_posts.csv'), encoding='ISO-8859-1', index=False)

In [31]:
df = pd.read_csv(os.path.join(filter_path, 'All_qualified_posts.csv'), encoding='ISO-8859-1')
df.groupby(['Question_valid_tags']).agg(['count'])

,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_date,Question_favorite_count,Question_last_edit_date,Question_score,Question_tags,Question_view_count,...,Owner_location,Owner_reputation,Owner_up_votes,Owner_down_votes,Owner_views,Answer_body,Answer_comment_count,Answer_creation_date,Answer_last_edit_date,Answer_score
,count,count,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
Question_valid_tags,,,,,,,,,,,,,,,,,,,,,
"['amazon-sagemaker', 'databricks']",2,2,2,2,2,1,0,2,2,2,...,1,2,2,2,2,1,1,1,0,1
"['amazon-sagemaker', 'mlflow']",16,16,16,16,16,4,4,16,16,16,...,6,16,16,16,16,2,2,2,0,2
['amazon-sagemaker'],2215,2215,2215,2215,2215,375,782,2215,2215,2215,...,951,2215,2215,2215,2215,734,734,734,143,734
['clearml'],40,40,40,40,40,0,24,40,40,40,...,13,40,40,40,40,20,20,20,7,20
['comet-ml'],10,10,10,10,10,4,6,10,10,10,...,4,10,10,10,10,4,4,4,3,4
"['databricks', 'databricks-unity-catalog']",7,7,7,7,7,0,4,7,7,7,...,1,7,7,7,7,3,3,3,0,3
"['databricks', 'mlflow']",65,65,65,65,65,6,29,65,65,65,...,27,65,65,65,65,13,13,13,5,13
['databricks'],5011,5011,5011,5011,5011,922,2612,5011,5011,5011,...,1944,5011,5011,5011,5011,1880,1880,1880,482,1880


In [32]:
df0 = df[pd.isna(df['Answer_score']) == False]
df0.to_csv(os.path.join(filter_path, 'All_qualified_completed_posts.csv'), encoding='ISO-8859-1', index=False)
df0.groupby(['Question_valid_tags']).agg(['count'])

,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_date,Question_favorite_count,Question_last_edit_date,Question_score,Question_tags,Question_view_count,...,Owner_location,Owner_reputation,Owner_up_votes,Owner_down_votes,Owner_views,Answer_body,Answer_comment_count,Answer_creation_date,Answer_last_edit_date,Answer_score
,count,count,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
Question_valid_tags,,,,,,,,,,,,,,,,,,,,,
"['amazon-sagemaker', 'databricks']",1,1,1,1,1,1,0,1,1,1,...,0,1,1,1,1,1,1,1,0,1
"['amazon-sagemaker', 'mlflow']",2,2,2,2,2,1,1,2,2,2,...,0,2,2,2,2,2,2,2,0,2
['amazon-sagemaker'],734,734,734,734,734,173,280,734,734,734,...,364,734,734,734,734,734,734,734,143,734
['clearml'],20,20,20,20,20,0,13,20,20,20,...,9,20,20,20,20,20,20,20,7,20
['comet-ml'],4,4,4,4,4,2,4,4,4,4,...,1,4,4,4,4,4,4,4,3,4
"['databricks', 'databricks-unity-catalog']",3,3,3,3,3,0,2,3,3,3,...,1,3,3,3,3,3,3,3,0,3
"['databricks', 'mlflow']",13,13,13,13,13,2,5,13,13,13,...,7,13,13,13,13,13,13,13,5,13
['databricks'],1880,1880,1880,1880,1880,426,1068,1880,1880,1880,...,834,1880,1880,1880,1880,1880,1880,1880,482,1880


In [33]:
df = pd.read_csv(os.path.join(filter_path, 'All_great_posts.csv'), encoding='ISO-8859-1')
df0 = df[pd.isna(df['Answer_score']) == False]
df0.to_csv(os.path.join(filter_path, 'All_great_completed_posts.csv'), encoding='ISO-8859-1', index=False)
df0.groupby(['Question_valid_tags']).agg(['count'])

,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_date,Question_favorite_count,Question_last_edit_date,Question_score,Question_tags,Question_view_count,...,Owner_location,Owner_reputation,Owner_up_votes,Owner_down_votes,Owner_views,Answer_body,Answer_comment_count,Answer_creation_date,Answer_last_edit_date,Answer_score
,count,count,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
Question_valid_tags,,,,,,,,,,,,,,,,,,,,,
"['amazon-sagemaker', 'databricks']",1,1,1,1,1,1,0,1,1,1,...,0,1,1,1,1,1,1,1,0,1
"['amazon-sagemaker', 'mlflow']",1,1,1,1,1,0,0,1,1,1,...,0,1,1,1,1,1,1,1,0,1
['amazon-sagemaker'],402,402,402,402,402,143,158,402,402,402,...,193,402,402,402,402,402,402,402,95,402
['clearml'],16,16,16,16,16,0,12,16,16,16,...,7,16,16,16,16,16,16,16,6,16
['comet-ml'],3,3,3,3,3,2,3,3,3,3,...,0,3,3,3,3,3,3,3,3,3
"['databricks', 'databricks-unity-catalog']",2,2,2,2,2,0,2,2,2,2,...,1,2,2,2,2,2,2,2,0,2
"['databricks', 'mlflow']",7,7,7,7,7,2,2,7,7,7,...,4,7,7,7,7,7,7,7,3,7
['databricks'],1183,1183,1183,1183,1183,360,709,1183,1183,1183,...,530,1183,1183,1183,1183,1183,1183,1183,312,1183


In [ ]:
page = -1
base_url = 'https://discuss.dvc.org/'
page_suffix = 'c/questions/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    if 'more_topics_url' not in topic_list.keys():
        break

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)


with open(os.path.join(raw_path, 'DVC.json'), 'w') as outfile:
    json_post_list = json.dumps(post_list, indent='\t')
    outfile.write(json_post_list)


In [ ]:
page = -1
base_url = 'https://community.wandb.ai/'
page_suffix = 'c/w-b-support/36.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    if 'more_topics_url' not in topic_list.keys():
        break

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

with open(os.path.join(raw_path, 'Weights & Biases.json'), 'w') as outfile:
    json_post_list = json.dumps(post_list, indent='\t')
    outfile.write(json_post_list)
